In [5]:
from pyqubo import Array, Constraint, Placeholder
import time
import numpy as np
import math

In [6]:
x = []
y = []
with open('../data/data2.txt','r') as file:
# with open('../coeff-generator/test_data.txt','r') as file:
    for line in file:
        cord = line.split()[:2]
        tx = float(cord[0])
        ty = float(cord[1])
        x.append(tx)
        y.append(ty)

In [7]:
ncity = len(x)
city_distance = np.zeros((ncity, ncity), dtype=np.float64)
for i in range(0, ncity):
    for j in range(i + 1, ncity):
        city_distance[i][j] = math.sqrt((x[i] - x[j]) ** 2 + (y[i] - y[j]) ** 2)
        city_distance[j][i] = city_distance[i][j]

In [55]:
n_city = ncity

t0 = time.time()
x = Array.create('c', (n_city, n_city), 'BINARY')

# Constraint not to visit more than two cities at the same time.
time_const = 0.0
for i in range(n_city):
    # If you wrap the hamiltonian by Const(...), this part is recognized as constraint
    time_const += Constraint((sum(x[i, j] for j in range(n_city)) - 1)**2, label="time{}".format(i))

# Constraint not to visit the same city more than twice.
city_const = 0.0
for j in range(n_city):
    city_const += Constraint((sum(x[i, j] for i in range(n_city)) - 1)**2, label="city{}".format(j))

# distance of route
distance = 0.0
for k in range(n_city):
    for i in range(n_city):
        for j in range(n_city):
            # we set the constant distance
            d_ij = city_distance[i][j]
            distance += 0.5 * d_ij * x[k, i] * (x[(k+1)%n_city, j] + x[(k-1)%n_city, j])

# Construct hamiltonian
A = Placeholder("A")
H = distance + A * (time_const + city_const)

# Compile model
t1 = time.time()
model = H.compile()
qubo, offset = model.to_qubo(index_label=True, feed_dict={"A": 2.0})
linear,quadratic,offset = model.to_ising(index_label=True, feed_dict={"A": 2.0})
t2 = time.time()

In [56]:
linear = dict(sorted(linear.items(), key=lambda x: x[0]))
with open('linear.txt','w') as file:
    for key,value in linear.items():
        file.write(str(key)+" "+str(value)+"\n")
linear

{0: 922.0647231072853,
 1: 922.0647231072853,
 2: 922.0647231072853,
 3: 922.0647231072852,
 4: 922.0647231072853,
 5: 922.0647231072853,
 6: 930.6817768903917,
 7: 930.6817768903916,
 8: 930.6817768903916,
 9: 930.6817768903917,
 10: 930.6817768903917,
 11: 930.6817768903917,
 12: 791.8354455699921,
 13: 791.8354455699922,
 14: 791.8354455699922,
 15: 791.8354455699922,
 16: 791.8354455699923,
 17: 791.8354455699924,
 18: 866.8202771376082,
 19: 866.8202771376082,
 20: 866.8202771376082,
 21: 866.8202771376081,
 22: 866.8202771376082,
 23: 866.8202771376082,
 24: 1693.0210516454845,
 25: 1693.0210516454845,
 26: 1693.0210516454845,
 27: 1693.0210516454847,
 28: 1693.0210516454845,
 29: 1693.0210516454845,
 30: 1120.5663900263744,
 31: 1120.5663900263744,
 32: 1120.5663900263744,
 33: 1120.5663900263744,
 34: 1120.5663900263744,
 35: 1120.5663900263744}

In [57]:
a = np.zeros((ncity**2, ncity**2))
for key, valuie in quadratic.items():
    a[key] = value
a

array([[   0.        , 1120.56639003, 1120.56639003, ...,    0.        ,
        1120.56639003,    0.        ],
       [   0.        ,    0.        , 1120.56639003, ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ..., 1120.56639003,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
        1120.56639003, 1120.56639003],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        , 1120.56639003],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ]])

In [58]:
quadratic = dict(sorted(quadratic.items(), key=lambda x: x[0]))
with open('quadratic.txt','w') as file:
    for key,value in quadratic.items():
        file.write(str(key[0])+" "+str(key[1])+" "+str(value)+"\n")

In [59]:
offset
with open('offset.txt','w') as file:
    file.write(str(offset)+"\n")

In [60]:
offset

18962.968993131424